<center><h1>Colocalization and Distance measurements of objects in fluorescence microscopy</h1></center>

<h2>Summary</h2>

This notebook will show how to obtain data related to colocalization and distance of segmented objects in images obtained using fluorescence microscopy. For the purpose of this notebook Zeiss CZI files obtained using ZEN for 3D Structured Illumination Microscopy (3D-SIM) are used. Dimensional order is TCZYX. Python-bioformats was used to extract image pixel values as a numpy array.

<h2>Imports</h2>

In [ ]:
import numpy as np
import SimpleITK as sitk

<h2>Retrieve Image Data</h2>

Included for the sake of completeness.
This can be skipped if your images are already sitk images.

In [ ]:
import javabridge
import bioformats as bf
import re
from xml.etree import ElementTree as ETree


filepath = "/path/to/your/file"

javabridge.start_vm(
        class_path=bf.JARS,
        run_headless=True
    )
md = bf.get_omexml_metadata(filepath)
rdr = bf.ImageReader(filepath)
mdroot = ETree.fromstring(re.sub(' xmlns="[^"]+"', '', md, count=1))
# Get metadata from image to be used in reading pixel data
main_metadata = dict(mdroot.find("Image").find("Pixels").items())
size = [int(item) for item in [main_metadata['SizeT'], 
                               main_metadata['SizeC'], 
                               main_metadata['SizeZ'], 
                               main_metadata['SizeY'], 
                               main_metadata['SizeX']]]
# Create empty array to be filled with pixel data.
image5d = np.empty(size)
# Fill image5d with pixel data extracted from image
for t in range(int(main_metadata['SizeT'])):
    for c in range(int(main_metadata['SizeC'])):
        for z in range(int(main_metadata['SizeZ'])):
            try:
                image5d[t,c,z] = rdr.read(c=c, z=z, t=t, rescale=False)
            except Exception as e:
                print("Error reading image: {}".format(e))
rdr.close()
# Remove dimensions with only one axis. In the test case it was the T dimension.
image5d = image5d.squeeze()
Blue_Channel = sitk.GetImageFromArray(image5d[0])
Red_Channel = sitk.GetImageFromArray(image5d[1])
Green_Channel = sitk.GetImageFromArray(image5d[2])

<h2>Segmenting Channels</h2>

In [ ]:
# Get connected components from pixels above a threshold value i.e. 20000
Blue_obj = sitk.ConnectedComponent(Blue_Channel>20000)
Red_obj = sitk.ConnectedComponent(Red_Channel>10000)
Green_obj = sitk.ConnectedComponent(Green_Channel>10000)

<h2>Bitwise Colocalization (Red/Green)</h2>

This determines which voxels contain both red and green objects.

In [ ]:
filt = sitk.AndImageFilter()
overlap_image = filt.Execute(Green_obj, Red_obj)
# Correcting for disjointed pixels that might be labelled as part 
# of the same colocalization object by the bitwise operation
overlap_image = sitk.ConnectedComponent(overlap_image)

<h2>Distance Measurements</h2>

//TODO
This will show how many of each object are found at a given distance.